In [2]:
import pandas as pd
import numpy as np

In [3]:
path = '../data/'
books = pd.read_csv(path + 'books_m1.csv')
users = pd.read_csv(path + 'users_m1.csv')
train = pd.read_csv(path + 'train_ratings.csv.bak')
test = pd.read_csv(path + 'test_ratings.csv.bak')
sub = pd.read_csv(path + 'sample_submission.csv.bak')

ratings = pd.concat([train, test, sub], axis=0)

### 인덱싱

### location 인덱스

In [4]:
locations = [i for i in users.columns if 'location_' in i]

In [5]:
users[locations] = users[locations].fillna('unknown')
users[locations]

,location_part1,location_part2,location_part3,location_part4,location_part5
0,unknown,unknown,timmins,ontario,canada
1,unknown,unknown,ottawa,ontario,canada
2,unknown,unknown,unknown,n/a,n/a
3,unknown,unknown,toronto,ontario,canada
4,unknown,unknown,victoria,british columbia,canada
...,...,...,...,...,...
68087,unknown,unknown,danville,pennsylvania,usa
68088,unknown,unknown,victoria,delaware,canada
68089,unknown,unknown,irvington,alabama,usa
68090,unknown,unknown,vancouver,washington,usa


In [6]:
users.to_csv('../data/users_m1.csv', index=False)
!cp ../data/users_m1.csv ../data/users.csv

In [7]:
test = test.merge(users[['user_id', *locations]], on='user_id', how='left')
train = train.merge(users[['user_id', *locations]], on='user_id', how='left')
sub = sub.merge(users[['user_id', *locations]], on='user_id', how='left')


In [8]:
train.to_csv('../data/train_ratings_m1.csv', index=False)
test.to_csv('../data/test_ratings_m1.csv', index=False)
sub.to_csv('../data/sample_submission_m1.csv', index=False)

In [9]:
!cp ../data/train_ratings_m1.csv ../data/train_ratings.csv
!cp ../data/test_ratings_m1.csv ../data/test_ratings.csv
!cp ../data/sample_submission_m1.csv ../data/sample_submission.csv

### user 수치

In [10]:
user_ratings = [i for i in users.columns if 'user_rating' in i]
user_ratings

['user_rating_count', 'user_rating_mode', 'user_rating_median']

In [11]:
users[user_ratings] = users[user_ratings].fillna(0)
users['user_rating_count'] = users['user_rating_count'].astype('int')
users['user_rating_mode'] = users['user_rating_mode'].astype('int')
users['user_rating_median'] = users['user_rating_median'].astype('int')
users[user_ratings]

,user_rating_count,user_rating_mode,user_rating_median
0,7,2,4
1,12,7,7
2,5520,8,7
3,7,7,7
4,120,8,8
...,...,...,...
68087,1,7,7
68088,1,8,8
68089,1,2,2
68090,1,10,10


In [12]:
max(users['user_rating_count'].values)

5520

In [13]:
users.info(0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68092 entries, 0 to 68091
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             68092 non-null  int64  
 1   location            68092 non-null  object 
 2   age                 40259 non-null  float64
 3   location_part1      68092 non-null  object 
 4   location_part2      68092 non-null  object 
 5   location_part3      68092 non-null  object 
 6   location_part4      68092 non-null  object 
 7   location_part5      68092 non-null  object 
 8   user_rating_count   68092 non-null  int64  
 9   user_rating_mode    68092 non-null  int64  
 10  user_rating_median  68092 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 5.7+ MB


In [14]:
users.to_csv('../data/users_m1.csv', index=False)
!cp ../data/users_m1.csv ../data/users.csv

In [15]:
test = test.merge(users[['user_id', *user_ratings]], on='user_id', how='left')
train = train.merge(users[['user_id', *user_ratings]], on='user_id', how='left')
sub = sub.merge(users[['user_id', *user_ratings]], on='user_id', how='left')

In [16]:
train.to_csv('../data/train_ratings_m1.csv', index=False)
test.to_csv('../data/test_ratings_m1.csv', index=False)
sub.to_csv('../data/sample_submission_m1.csv', index=False)

In [17]:
!cp ../data/train_ratings_m1.csv ../data/train_ratings.csv
!cp ../data/test_ratings_m1.csv ../data/test_ratings.csv
!cp ../data/sample_submission_m1.csv ../data/sample_submission.csv

In [18]:
# !conda install scipy -y
# !conda install scikit-learn -y

In [26]:
from sklearn.model_selection import train_test_split

seed=42

X_train, X_test_tmp, y_train, y_test_tmp = train_test_split(ratings[['user_id', 'isbn']], ratings['rating'], test_size=0.4, random_state=seed)
X_valid, X_test, y_valid, y_test = train_test_split(X_test_tmp, y_test_tmp, test_size=0.5, random_state=seed)

In [30]:
def df_to_arr(X: pd.DataFrame, y: pd.Series, arr: np.ndarray) -> np.ndarray:
  for i, value in enumerate(X.values):
    arr[value[0], value[1]] += y.values[i]

  return arr

train = np.zeros((len(user2idx), len(book2idx)))
valid = np.zeros((len(user2idx), len(book2idx)))
test = np.zeros((len(user2idx), len(book2idx)))

train = df_to_arr(X_train, y_train, train)
valid = df_to_arr(X_valid, y_valid, valid)
test = df_to_arr(X_test, y_test, test)

In [34]:
from scipy.sparse import csr_matrix, linalg

def truncated_svd(mat: np.ndarray, k: int=100):
  u, s, vh = linalg.svds(csr_matrix(train), k)
  return u @ np.diag(s) @ vh

In [35]:
svd_output2 = truncated_svd(train, k=2)
svd_output5 = truncated_svd(train, k=5)
svd_output10 = truncated_svd(train, k=10)

: 

In [ ]:
def modify_range(rating: float) -> float:
  if rating < 0:
    return 0
  elif rating > 10:
    return 10
  else:
    return rating
  
def matrix_rmse(real_mat: np.ndarray, predict_mat: np.ndarray) -> float:
  indicator = np.stack(real_mat.nonzero(), axis=1)

  cost = 0
  for i, ind in enumerate(indicator):
    pred = predict_mat[ind[0], ind[1]]
    real = real_mat[ind[0], ind[1]]
    cost += pow(real - modify_range(pred), 2)

  return np.sqrt(cost/ len(indicator))

In [ ]:
print("k=2 : ", matrix_rmse(test, svd_output2))
print("k=5 : ", matrix_rmse(test, svd_output5))
print("k=10 : ", matrix_rmse(test, svd_output10))